### TASK

    1. new UI design
    2. the 3 predefined sets parameter autosetting by clicking on "fast/median/slow mode" buttons in advance 
    3. auto-select the ROI(region of interested) and crop
    4. write function for the error message output
    5. refactor the code into modularization

In [3]:
from PyQt5.QtCore import * 
from PyQt5.QtGui import *  
from PyQt5.QtWidgets import *  

import pyqtgraph as pg   ##
import pyqtgraph as ptime
import cv2
import time
import math
from datetime import datetime
import os
from pathlib import Path
from pandas import DataFrame
import csv

# must import the numpy befor ethe PySpin
import numpy as np
import PySpin

system = PySpin.System.GetInstance()
version = system.GetLibraryVersion()
print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))
# Library version: 1.20.0.15
### ----- Parameters to Change ----- ###
win_length = 1375  #Divisible by 6 as the ratio of panel to view window is 1:5
win_width = 750
win_scale = False
no_cancel = False
cwDir = 0
file_stem = 'Name Your File'
filename = 'file.save'
console_log = []
console_num = 0
filelist = [0]
got_camera = False
roi_mode = 0
view_mode = 0
save_mode = 0


roi_x, roi_y , roi_width, roi_height = (0, 0, 0, 0)
capture_x,capture_y, capture_width, capture_height = (0, 0, 0, 0)

exposure_time = 100       #micro seconds min=10us (FIXED)
FPS = 25                    #Frame rate to capture 
gain = 0                   #Sensor gain (FIXED)
image_format = 'Mono8'      #image pixel format. 'Mono8' or 'Bayer' for BGR
width = 1440               #image width in pixels (FIXED)
height = 600                #image height in pixels (FIXED)
Show = 1                    #Show = show the video while loop
Skip_frames = 25            #number of frames to skip before imshow
num_frames = 1000            #number of frames to view
video_record = 0            #1: to record the video, 0: No Record
video_mode = 'MJPG'          #AVI or MJPG 
video_name = 'zdddd.avi'    #Filename to save as

#angle variables
angle = 0                   #rotational angle
rot_x = 0                   #x pos value of final rotated and cropped image
rot_y = 0                   #y pos value of final rotated and cropped image
rot_width = 0               #width of final rotated and cropped image
rot_height = 0              #height of final rotated and cropped image


font = cv2.FONT_HERSHEY_SIMPLEX

H = 110             #No. of pixels to select for height of Region Of Interest
blur_value = 3      #value = 3,5 or 7 (ODD).median Blur value determines the accuracy of detection
Delay = 1           #time value in miliseconds. (Fastest) Minimum = 1ms
Show = 1            #To display the image. 1 = On, 0 = Off
Skip_frames = 10     #number of frames to skip before Im showing
Channels = 35       #number of Channels
Export_Results = 1  #export data to excel

### ----- Initialisation ----- ###
mask = cv2.createBackgroundSubtractorMOG2(history = 3,
                                          varThreshold = 100,
                                          detectShadows = False)
sum_ch1 = [0]*Channels
cwDir = os.getcwd()

Library version: 1.29.0.5


## Functions

In [4]:
'''
Global parameters used, making classes or setter/getter would be better
'''
### ----- Parameters to Change ----- ###
def reset_parameters():
    global FPS, exposure_time, num_frames, gain, width, height
    FPS = 25
    exposure_time = 100000000
    num_frames = 250
    gain = 0
    width = 1440
    height = 1080

    cam_field2.setText(str(FPS))
    cam_field3.setText(str(exposure_time))
    cam_field4.setText(str(num_frames))
    cam_field5.setText(str(gain))
    cam_field6.setText(str(width))
    cam_field7.setText(str(height))
    return

def cancel():
    global no_cancel
    if no_cancel == True:
        no_cancel = False
        console_print('CANCEL button pressed')
        camera_reset()
    else: return

def rotation_value():
    global angle
    angle = (slide1.value()/5)
    console_print('The rotation angle is '+str(angle))
    return angle

def showdialog():
    msg = QMessageBox()
    msg.setIcon(QMessageBox.Information)
    msg.setText("Saved Parameters")
    msg.setInformativeText("H = %i, Ch = %i, Blur = %i, Delay = %ims, Skip_frames = %i. Show = %i"%(H,Channels, blur_value,Delay,Skip_frames,Show))
    msg.setWindowTitle("Saving Parameters")
    msg.setStandardButtons(QMessageBox.Ok)
    msg.exec_()
    return

def select_folder():
    global cwDir
    cwd = os.getcwd()
    cwDir = QtGui.QFileDialog.getExistingDirectory(None, 
                                                'Open working directory', 
                                                cwd, 
                                                QtGui.QFileDialog.ShowDirsOnly)
    cam_field1.setText(cwDir)

def select_files():
    global filelist
    options = QtGui.QFileDialog.Options()
    options |= QtGui.QFileDialog.DontUseNativeDialog
    
    filelist, suff = QtGui.QFileDialog.getOpenFileNames(None,
                                                        "LALALALALAL",
                                                        "",
                                                        "video (*.avi)", #"video (*.avi);;All Files (*)"
                                                        options=options)
    if not(filelist):
        console_print('Error Loading | No files selected')
        field1.setText('No File(s) Selected')
        print(filelist)
        return
    
    file = Path(filelist[0])
    field1.setText(str(file))
    if len(filelist) == 1:
        console_print('Selected: ' + str(filelist[0]))
    elif len(filelist) > 1:
        console_print('Selected: ' + str(len(filelist)) + ' Files')

    return

# prints DMY on console
##########################################
# IMAGE CAPTURING
##########################################

def camera_start():
    global got_camera, cam, cam_list

    if got_camera == True: return
    start_time = time.time()
    ## --- Setup Camera --- ##
    # [GET] system
    system = PySpin.System.GetInstance()

    try:
        # [GET] camera list
        cam_list = system.GetCameras()
        # Init_camera
        cam = cam_list.GetByIndex(0)
    except:
        got_camera = False
        return

    # [Initialize] camera
    cam.Init()
    cam.BeginAcquisition()
    cam.EndAcquisition()
    num_cameras = cam_list.GetSize()
    print('[INFO] Number of cameras detected: %d' % num_cameras)
    # [LOAD] default configuration
    cam.UserSetSelector.SetValue(PySpin.UserSetSelector_Default)
    cam.UserSetLoad()
    got_camera = True

    return

def camera_reset():
    cam.OffsetY.SetValue(0)
    cam.OffsetX.SetValue(0)
    return

def camera_properties(c_width, c_height, c_fps):
    global exposure_time, width, height
    width = c_width
    height = c_height
    # [SET] Image dimenstion width and height and center ROI
    max_width = cam.Width.GetMax()
    max_height = cam.Height.GetMax()
    if width <= max_width and width >= 100:
        if width%4 != 0:
            print(width, 'not divisible by 4')
            width=width-width%4
    elif width < 100 and width > 0:
        width = 100
    else: width = max_width
    cam_field6.setText(str(width))
    
    if height <= max_height and height >= 100:
        if height%2 != 0:
            print(height, 'not divisible by 2')
            height=height+1
    elif height < 100 and height > 0:
        height = 100
    else: height = max_height
    cam_field7.setText(str(height))
    cam.Width.SetValue(width)
    cam.Height.SetValue(height)
    cam.OffsetY.SetValue(round((max_height-height)/2)-round((max_height-height)/2)%2)
    cam.OffsetX.SetValue(round((max_width-width)/2)-round((max_width-width)/2)%4)
    print('[INFO] Frame Width =',width,'| Height =',height)

    # [SET] analog. Set Gain. Turn off Gamma.
    gain = cam_field5.text()
    cam.GainAuto.SetValue(PySpin.GainAuto_Off)
    cam.Gain.SetValue(int(gain))
    cam.GammaEnable.SetValue(False)
    print('Cam Properties gain End')
    
    # [SET] acquisition. Continues acquisition. Auto exposure off. Set frame rate. 
    exposure_time = cam_field3.text()
    cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
    cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
    exposure_time = min(cam.ExposureTime.GetMax(), int(exposure_time))
    cam.ExposureMode.SetValue(PySpin.ExposureMode_Timed)
    cam.ExposureTime.SetValue(int(exposure_time))
    print('Cam Properties exposure tim End')

    # [SET] Framerates
    cam.AcquisitionFrameRateEnable.SetValue(True)
    max_FPS = cam.AcquisitionFrameRate.GetMax()
    try: int(c_fps)
    except:    c_fps = 25
    if int(c_fps) > (round(max_FPS)-1):
        c_fps = (round(max_FPS)-1)
    elif int(c_fps) >= 1 and int(c_fps)<= (round(max_FPS)-1):
        pass
    else: c_fps = 25
    
    cam.AcquisitionFrameRate.SetValue(int(c_fps))
    print('Cam Properties End')
    return

def video():
## cv2 benchmark
    global H, Channels, blur_value, Delay, Skip_frames, Show, no_cancel, angle

    try:
        H = round(float(field2.text()))
        Channels = round(float(field3.text()))
        blur_value = round(float(field4.text()))
        Delay = round(float(field5.text()))
        Skip_frames = round(float(field6.text()))
        if chk_box1.isChecked():
            Show = 1
        else:
            Show = 0
            btn2.setEnabled(False)
    except:
        console_print('There is an error in the field values')
        H = 110             #No. of pixels to select for height of Region Of Interest
        blur_value = 3      #value = 3,5 or 7 (ODD).median Blur value determines the accuracy of detection
        Delay = 1           #time value in miliseconds. (Fastest) Minimum = 1ms
        Show = 1            #To display the image. 1 = On, 0 = Off
        Skip_frames = 10     #number of frames to skip before Im showing
        Channels = 35       #number of Channels

    if H < 10:
        H = 10
        field2.setText(str(10))
        
    if Channels < 2:
        Channels = 2
        field3.setText(str(2))

    if (blur_value%2 == 0 or blur_value < 3) or blur_value > 15:
        blur_value = 3
        field4.setText(str(3))
        
    if Delay <1:
        Delay = 1
        field5.setText(str(1))
        
    if Skip_frames <1:
        Skip_frames = 1
        field6.setText(str(1))
        
    if (not(filelist) or filelist[0] == 0):
        return console_print('No File Selected')
    
    no_cancel = True
    
    width = w.frameGeometry().width()
    height = w.frameGeometry().height()

    btn1.setEnabled(False)
    btn4.setEnabled(False)
    
    cv = 0
    name = Path(filelist[0])
    
    cap = cv2.VideoCapture(filelist[0])
    ret, frame = cap.read()
    size = frame.shape
    x1 = size[1]
    y1 = size[0]
    step = frame.size / (size[0]) / 3
    qformat = QImage.Format_Indexed8
    
    console_print('Playing File: ' + str(filelist[0]))
    bar1.setMaximum(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap = cv2.VideoCapture(filelist[0])
    start= time.time()
    
    while(cap.isOpened() and no_cancel):
        ret, frame = cap.read()
        
        if not ret: break
       #blur so that noise isnt detected.
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        crop = mask.apply(frame)
        crop = cv2.medianBlur(frame,blur_value)
        crop = cv2.threshold(crop, 125, 255, cv2.THRESH_BINARY)[1]
        
        if chk_box1.isChecked() and cv%Skip_frames == 0:
            if chk_box2.isChecked():
                frame = cv2.flip(frame, -1)
            img = QImage(frame, size[1], size[0], step, qformat)
            img = QPixmap.fromImage(img)
            rotate = QTransform().rotate(angle)
            img = img.transformed(rotate, Qt.SmoothTransformation)
            rad = math.radians(abs(angle))
            cos = math.cos(rad)
            sin = math.sin(rad)
            x2 = (x1*cos) + (y1*sin)
            rot_width = x1 - abs(x2-x1)
            rot_height = (y1 * cos) - (x1 * sin)
            img = img.copy( abs(x2-x1), abs(x1*sin), int(rot_width), int(rot_height))    # copy(int x, int y, int width, int height)
            if chk_box3.isChecked():
                img=img.scaled(round(width*(5/6))-50, height-25, aspectRatioMode = Qt.KeepAspectRatio)
            label.setPixmap(img)
            cv2.waitKey(Delay) 
            
        cv += 1
        bar1.setValue(cv)
    print(abs(x2-x1), abs(x1*sin), int(rot_width), int(rot_height))
    end = time.time()
    label.setText("Hello...What's next?")
    #cv2.destroyAllWindows()
    console_print("H = %i, Ch = %i, Blur = %i, Delay = %ims, Skip_frames = %i. Show = %i"%(H,Channels, blur_value,Delay,Skip_frames,Show))
    console_print('Opencv2 Processing FPS = %0.3f'%(cv/(end-start)))
    console_print('Total number of frames = ' + str(cv))
    no_cancel = False
    btn1.setEnabled(True)
    btn2.setEnabled(True)
    btn4.setEnabled(True)
    return

def camera_view():
    global no_cancel, got_camera, frame, angle
    global view_mode, save_mode
    global rot_x, rot_y, rot_width, rot_height
    start_time = time.time()
    # [GET] camera list
    cam_list = system.GetCameras()
    print(cam_list)
    num_cameras = cam_list.GetSize()
    print('num cams',num_cameras)
    camera_start()
    end_time = time.time()
    print("Benchmark:",end_time - start_time)
    if got_camera == True and num_cameras == 0:
        print('trouble')
        console_print('The camera was disconnected')
        got_camera = False
        return
            
    if got_camera == False:
        console_print('No Detectable Camera')
        return
    btn5.setEnabled(False)
    btn4.setEnabled(False)
    btn1.setEnabled(False)
    btn3.setEnabled(False)
    no_cancel = True
    
    console_print('Camera View BEGIN')
    print('Initial FPS view = ',cam.AcquisitionFrameRate.GetValue())
    c_width = cam_field6.text()
    c_height = cam_field7.text()
    camera_properties(int(c_width) , int(c_height), 25)
    cam.BeginAcquisition()
    count = 0
    view_mode = 1
    save_mode == 0
    local_angle = (angle+1)   #arbitury number assigned
    ch = int(field3.text())
    sub_ch = []
    
    while no_cancel:
        image_primary = cam.GetNextImage()
        frame = np.array(image_primary.GetNDArray())
        if chk_box2.isChecked():
            frame = cv2.flip(frame, -1)

        if (local_angle != angle):
            rad = math.radians(abs(angle))
            cos = math.cos(rad)
            sin = math.sin(rad)
            y1,x1 = frame.shape
            image_center = (x1 / 2, y1 / 2)
            y2 = (y1 * cos) + (x1 * sin)
            x2 = (x1 * cos) + (y1 * sin)
            rot_x = int(abs(x2-x1))
            rot_y = int(abs(x1*sin))
            rot_width = int(x1 - abs(x2-x1))
            rot_height = int((y1 * cos) - (x1 * sin))
            M = cv2.getRotationMatrix2D((image_center),angle,1)
            M[0, 2] += ((x2 / 2) - image_center[0])
            M[1, 2] += ((y2 / 2) - image_center[1])
                      
            print('angle has changed')
            local_angle = angle
        
        frame = cv2.warpAffine(frame,M,(int(x2),int(y2)))  
        frame_crop = frame[rot_y:(rot_y+rot_height) , rot_x:(rot_x+rot_width)]  #img[y:y+h, x:x+w]
        
        
        if chk_box4.isChecked() and (capture_width+capture_height) > 0:
            cv2.putText(frame_crop, 'ROI', (roi_x + 10, roi_y + 30), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame_crop, 'Capture Size', (capture_x + 10, capture_y + 30), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.rectangle(frame_crop, (roi_x,roi_y), (roi_x+roi_width, roi_y+roi_height), (255,255,255), 3,1,0)
            cv2.rectangle(frame_crop, (capture_x ,capture_y), (capture_x + capture_width, capture_y + capture_height), (255,255,255), 3,1,0)
            # draw lines on crop frame
            ## 
            for x in range(ch):
                sub_ch_x = round(x*(roi_width/(ch)))
                sub_ch.append(sub_ch_x)
                cv2.putText(frame_crop, str(x+1), ((roi_x+sub_ch[x]+10), (roi_y + int(roi_width*110/840) - 25)), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.line(frame_crop, ((roi_x+sub_ch[x]),roi_y), ((roi_x+sub_ch[x]), (roi_y+int(roi_width*110/840))), (200,200,100),2) 

        size = frame.shape
        step = frame.size / (size[0])
        
        qformat = QImage.Format_Indexed8
        img = QImage(frame, size[1], size[0], step, qformat)
        img = img.copy( rot_x, rot_y, rot_width, rot_height)    # copy(int x, int y, int width, int height)
        img = QPixmap.fromImage(img)
        
        # img show
        if chk_box3.isChecked():
            img=img.scaled(round(width*(0.6))-50, round(height*(0.6))-25, aspectRatioMode = Qt.KeepAspectRatio)
        label.setPixmap(img)
        cv2.waitKey(Delay)
        image_primary.Release()

    cam.EndAcquisition()
    print('FPS view = ',cam.AcquisitionFrameRate.GetValue())
    view_mode = 0
    label.setText("Hello...What's next?")
    camera_reset()
    no_cancel = False
    
    btn1.setEnabled(True)
    btn4.setEnabled(True)
    btn3.setEnabled(True)
    btn5.setEnabled(True)

    return

def roi_select():
    global roi_mode
    global roi_x, roi_y, roi_width, roi_height
    global capture_x, capture_y, capture_width, capture_height
    if no_cancel == True:
        print('camera is on')
        cap = cv2.VideoCapture(0)
        ret,frame = cap.read()

        if chk_box2.isChecked():
            #frame_error here
            frame = cv2.flip(frame, -1)
        print(2)
##        cv2.putText(frame_crop, 'Please select a region of interest and press ENTER', (30, 30), font, 1, (2, 255, 255), 1, cv2.LINE_AA)

        sel = cv2.selectROI(frame)
        print(3)
        cv2.destroyAllWindows()
        chk_box4.setCheckState(True)
        
    else:
        print('can capture')
        camera_start()
        camera_reset()
        c_width = cam_field6.text()
        c_height = cam_field7.text()
        camera_properties(int(c_width) , int(c_height), 25)
        cam.BeginAcquisition()
        roi_image = cam.GetNextImage()
        roi_image = np.array(roi_image.GetNDArray())

    if angle != 0:
        rad = math.radians(abs(angle))
        cos = math.cos(rad)
        sin = math.sin(rad)
        y1,x1 = roi_image.shape
        image_center = (x1 / 2, y1 / 2)
        y2 = (y1 * cos) + (x1 * sin)
        x2 = (x1 * cos) + (y1 * sin)
        rot_x = int(abs(x2-x1))
        rot_y = int(abs(x1*sin))
        rot_width = int(x1 - abs(x2-x1))
        rot_height = int((y1 * cos) - (x1 * sin))
        M = cv2.getRotationMatrix2D((image_center),angle,1)
        M[0, 2] += ((x2 / 2) - image_center[0])
        M[1, 2] += ((y2 / 2) - image_center[1])
                  
        print('angle has changed')
        local_angle = angle
            
        roi_image = cv2.warpAffine(roi_image,M,(int(x2),int(y2)))  
        roi_image = roi_image[rot_y:(rot_y+rot_height) , rot_x:(rot_x+rot_width)]  #img[y:y+h, x:x+w]
        
    if chk_box2.isChecked():
        roi_image = cv2.flip(roi_image, -1)
    cv2.putText(roi_image, 'Please select a region of interest and press ENTER', (30, 30), font, 1, (2, 255, 255), 1, cv2.LINE_AA)
    cam.EndAcquisition()
    sel = cv2.selectROI(roi_image)
    cv2.destroyAllWindows()
    
    chk_box4.setCheckState(True)
    print('Cam Acuire end')
    #roi_x , y from top left corner
    roi_x, roi_y, roi_width, roi_height = sel

    ## determine Capture X parameters
    if roi_x <= 50:
        capture_x = 0
        if roi_width + roi_x <= 1340:
            capture_width = roi_width+50 + roi_x
        else: capture_width = 1440
    elif roi_x + roi_width >= 1340:
        capture_x = roi_x - 50
        capture_width = (1440 - capture_x)
    elif roi_width >= 1340:
        capture_x = 0
        capture_width = 1440
    else:
        capture_x = roi_x - 50
        capture_width = roi_width + 100

    # determine capture Y parameters
    if roi_y <= 80:
        capture_y = 0
        if roi_height + roi_y <= 920:
            capture_height = roi_height+80+roi_y
        else: capture_height = 1080
    elif roi_y +roi_height >= 920:
        capture_y = roi_y - 80
        capture_height = (1080 - capture_y)
    elif roi_height >= 920:
        capture_y = 0
        capture_height = 1080
    else:
        capture_y = roi_y - 80
        capture_height = roi_height + 160
    
    print(sel)
    print(capture_x, capture_y, capture_width, capture_height)
    return

def save_data():
    global view_mode, no_cancel, save_mode
    global got_camera, FPS, angle
    global rot_x, rot_y, rot_width, rot_height

    local_angle = (angle+1)
    
    # [GET] camera list
    cam_list = system.GetCameras()
    print(cam_list)
    num_cameras = cam_list.GetSize()
    print('num cams',num_cameras)
    camera_start()
    if got_camera == True and num_cameras == 0:
        print('trouble')
        console_print('The camera was disconnected')
        got_camera = False
        return

    if rot_width == 0:
        console_print('Plese click on "CAMERA" and confirm the field of view')
        return
    elif roi_width == 0:
        console_print('Plese Select "ROI" and confirm number of channels')
        return        
            
    if got_camera == False:
        console_print('No Detectable Camera')
        return

    print(rot_x, rot_y, rot_width, rot_height)
    print(roi_x, roi_y, roi_width, roi_height)
    
    btn4.setEnabled(False)
    btn1.setEnabled(False)
    no_cancel = True
    FPS = int(cam_field2.text())
    print(FPS)    
    console_print('Camera View BEGIN')
    
    c_width = int(cam_field6.text())
    c_height = int(cam_field7.text())
    
    print(c_width,c_height)
    camera_properties(c_width , c_height, FPS)
    print('Initial FPS view = ',cam.AcquisitionFrameRate.GetValue())
    cam.BeginAcquisition()
    count = 0
    view_mode = 1
    save_mode == 0
    bar1.setMaximum(int(cam_field4.text()))
    mask = cv2.createBackgroundSubtractorMOG2(history = 3,
                                                varThreshold = 100,
                                                detectShadows = False)    
    ch = int(field3.text())
    sub_ch = []
    sum_ch1 = [0]*ch
    
    file_stem = cam_field1_1.text()
    cur_dir = Path(cwDir)
    cur_file = cur_dir/file_stem
##    suffix = '.avi'
##    out = cv2.VideoWriter(str(Path(str(cur_file)+suffix)), cv2.VideoWriter_fourcc(*'MJPG'), 25.0, (c_width, c_height), isColor=False)


    if (local_angle != angle):
        rad = math.radians(abs(angle))
        cos = math.cos(rad)
        sin = math.sin(rad)
        
        #original frame capture dimensions
        y1 = c_height
        x1 = c_width
        image_center = (x1 / 2, y1 / 2)
        
        #rotated array dimensions
        y2 = (y1 * cos) + (x1 * sin)
        x2 = (x1 * cos) + (y1 * sin)
        rot_x = int(abs(x2-x1))
        rot_y = int(abs(x1*sin))
        rot_width = int(x1 - abs(x2-x1))
        rot_height = int((y1 * cos) - (x1 * sin))

        
        #Maxtrix transformation
        M = cv2.getRotationMatrix2D((image_center),angle,1)
        M[0, 2] += ((x2 / 2) - image_center[0])
        M[1, 2] += ((y2 / 2) - image_center[1])
        
                  
        print('angle has changed')
        local_angle = angle


    ## assigning subchannels - sub_ch
    for x in range(ch+1):
        sub_ch_x = round(x*(roi_width/(ch)))
        sub_ch.append(sub_ch_x)


    # initialization of VideoWriter
    if menu1.currentText() == 'AVI':
        suffix = '.avi'
        out = cv2.VideoWriter(str(Path(str(cur_file)+suffix)), 0x00000000, 25.0, (rot_width, rot_height), isColor=False)
    elif menu1.currentText() == 'MJPG':
        suffix = '.avi'
        out = cv2.VideoWriter(str(Path(str(cur_file)+suffix)), cv2.VideoWriter_fourcc(*'MJPG'), 25.0, (rot_width, rot_height), isColor=False)
    elif menu1.currentText() == 'MP4':
        suffix = '.mp4'
        out = cv2.VideoWriter(str(Path(str(cur_file)+suffix)), 0x00000022, 25.0, (rot_width, rot_height), isColor=False)
    else:
        suffix = '.avi'
        out = cv2.VideoWriter(str(Path(str(cur_file)+suffix)), cv2.VideoWriter_fourcc(*'MJPG'), 25.0, (rot_width, rot_height), isColor=False)
    filename = Path(str(cur_file)+suffix)
    console_print('The saved file name = '+str(filename))
    

    while (no_cancel and count <= int(cam_field4.text())):
    
        count +=1
        image_primary = cam.GetNextImage()
        frame = np.array(image_primary.GetNDArray())
        
        if chk_box2.isChecked():
            frame = cv2.flip(frame, -1)

        if angle != 0:    
            frame = cv2.warpAffine(frame,M,(int(x2),int(y2)))  
            frame_crop = frame[rot_y:(rot_y+rot_height) , rot_x:(rot_x+rot_width)]  #img[y:y+h, x:x+w]

        if angle == 0:
            frame_crop = frame
        out.write(frame_crop)

        frame_count = frame_crop[roi_y:(roi_y + int(roi_width*110/840)) , roi_x:(roi_x + roi_width)]
        frame_mask = mask.apply(frame_count)
        frame_mask = cv2.medianBlur(frame_mask,blur_value)
        frame_mask = cv2.threshold(frame_mask, 125, 255, cv2.THRESH_BINARY)[1]

        # find contours
        contours, hierarcy = cv2.findContours(frame_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # list of all the coordinates (tuples) of each cell
        coord_list = []

        # to find the coordinates of the cells
        for i in range(len(contours)):
            avg = np.mean(contours[i], axis = 0)
            coord = (int(avg[0][0]), int(avg[0][1])) ##Coord is (y,x)
##            if Show == 1:
##                cv2.circle(frame_count, coord, 10, (255, 0, 255), 1)
            ch_pos = int(math.floor((coord[0])/sub_ch[1]))
            try:
                sum_ch1[ch_pos] += 1
            except:
                #Undeclared variable?
                error += 1
        
        if FPS < 26 or (count%(int(FPS/5)) == 0):

            if chk_box4.isChecked() and (capture_width+capture_height) > 0:
                cv2.putText(frame_crop, 'ROI', (roi_x + 10, roi_y + 30), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame_crop, 'Capture Size', (capture_x + 10, capture_y + 30), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame_crop, (roi_x,roi_y), (roi_x+roi_width, roi_y+roi_height), (255,255,255), 3,1,0)
                cv2.rectangle(frame_crop, (capture_x ,capture_y), (capture_x + capture_width, capture_y + capture_height), (255,255,255), 3,1,0)
                # draw lines on crop frame
                ## 
                for x in range(ch):
                    cv2.putText(frame_crop, str(x+1), ((roi_x+sub_ch[x]+10), (roi_y + int(roi_width*110/840) - 25)), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    cv2.line(frame_crop, ((roi_x+sub_ch[x]),roi_y), ((roi_x+sub_ch[x]), (roi_y+int(roi_width*110/840))), (200,200,100),2)
                
            qformat = QImage.Format_Indexed8
            img = QImage(frame, int(x2), int(y2), int(x2), qformat)
            img = img.copy( rot_x, rot_y, rot_width, rot_height)    # copy(int x, int y, int width, int height)
            img = QPixmap.fromImage(img)
            
            if chk_box3.isChecked():
                img=img.scaled(round(width*(0.6))-50, round(height*(0.6))-25, aspectRatioMode = Qt.KeepAspectRatio)
            
            cv2.imshow('frame_count', frame_count)
            
            label.setPixmap(img)
            cv2.waitKey(Delay)
            
        image_primary.Release()
        bar1.setValue(count)


    out.release    
    cam.EndAcquisition()
    cv2.destroyAllWindows()
    print('FPS captured is =' , FPS)
    view_mode = 0
    label.setText("Hello...What's next?")
    camera_reset()
    no_cancel = False
    #set an array of sub channel dimension
    print('[RESULTS] for RUN is ', sum_ch1)
    total_sum = []
    
    total_sum.append(sum_ch1)
    
    ###write dataframes and export to an Excel file    
    check = 0
    title = []
    for j in range(len(total_sum)):
        if check < len(total_sum[j]):check = len(total_sum[j])
        title.append('Run %i '%(j+1)+str(cur_file))

    index=np.arange(0,check,1)

    for k in range(len(total_sum)):
        if len(total_sum[k]) < check:
            for l in range(len(total_sum[k]),check):
                total_sum[k].append(0)

    TTotal_sum = list(map(list, zip(*total_sum)))
    df = DataFrame(data=TTotal_sum, columns = title)
    df.to_excel(str(Path(str(cur_file)+ ".xlsx")), index=False, sheet_name="Results")

    btn1.setEnabled(True)
    btn4.setEnabled(True)     
    log_param()
    
    return

def console_print(string):
    # dd/mm/YY H:M:S
    global console_num
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    log_input = (now, string)
    console_num += 1
    console_value = (str(console_num) + ' | ' + dt_string + ' |   ' + str(string))
    console_list.insertItem(0,console_value)
    console_log.append(log_input)
    
def save():
    global filename, file_stem
    file_stem = cam_field1_1.text()
    cur_dir = Path(cwDir)
    cur_file = cur_dir/file_stem
    if menu1.currentText() == 'AVI':
        suffix = '.avi'
    elif menu1.currentText() == 'MJPG':
        suffix = '.avi'
    elif menu1.currentText() == 'MP4':
        suffix = '.mp4'
    else: suffix = '.avi'
    filename = Path(str(cur_file)+suffix)
    console_print('The saved file name = '+str(filename))    
    
def log_param():
    from datetime import date
    from datetime import datetime

    today = date.today()
    date = today.strftime("%B %d, %Y")
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    file_stem = cam_field1_1.text()
    cur_dir = Path(cwDir)
    cur_file = cur_dir/file_stem

    log=[]
    log_param_name = [cam_name1.text(), cam_name1_1.text(), cam_name1_2.text(),
                      cam_name2.text(), cam_name3.text(), cam_name4.text(),
                      cam_name5.text(), cam_name6.text(),
                      cam_name7.text(), cam_name9.text(),
                      name1.text(), name2.text(), name3.text(), 
                      name4.text(), name5.text(), name6.text()]
    log_param_value = [cam_field1.text(), cam_field1_1.text(), menu1.currentText(),
                       cam_field2.text(), cam_field3.text(), cam_field4.text(),
                       cam_field5.text(), cam_field6.text(),
                       cam_field7.text(), rotation_value(),
                       field1.text(), field2.text(), field3.text(),
                       field4.text(), field5.text(), field6.text() ]
    log.append(log_param_name)
    log.append(log_param_value)

    with open(str(Path(str(cur_file) + "_log" + ".txt")), 'w') as f:
        f.write(date + "\n")
        f.write(dt_string + "\n")
        f.write("===================================="+"\n")
        for x in range(len(log_param_name)):
            f.write(str(log_param_name[x]) + " " + str(log_param_value[x]) +"\n")
        f.write("===================================="+"\n")
    return

def predefined_sets():
    
    field2.setText(str(10))

    
    
############################################################################    
play_btn = ("QPushButton { background-color: rgb(25,185,75); font: bold 12px;}"
            "QPushButton { border-color: rgb(40,40,15); border-width: 2px; }"
            "QPushButton { border-style: inset; border-radius: 4px; padding: 10px; }"
            "QPushButton:pressed { background-color: rgb(50,165,75) }")

cancel_btn = ("QPushButton { background-color: rgb(185,25,75); font: bold 12px;}"
            "QPushButton { border-color: rgb(40,40,15); border-width: 2px; }"
            "QPushButton { border-style: inset; border-radius: 4px; padding: 10px; }"
            "QPushButton:pressed { background-color: rgb(165,50,75) }")

save_btn = ("QPushButton { background-color: rgb(25,120,250); font: bold 12px;}"
            "QPushButton { border-color: rgb(40,40,15); border-width: 2px; }"
            "QPushButton { border-style: inset; border-radius: 4px; padding: 10px; }"
            "QPushButton:pressed { background-color: rgb(25,120,200) }")

cam_btn = ("QPushButton { background-color: rgb(200,200,200); font: bold 12px;}"
            "QPushButton { border-color: rgb(40,40,15); border-width: 2px; }"
            "QPushButton { border-style: inset; border-radius: 4px; padding: 10px; }"
            "QPushButton:pressed { background-color: rgb(180,180,180) }")


## --- Setup Camera --- ##
#Try using camera trigger instead and see if it speeds up the image capture
# [GET] system
system = PySpin.System.GetInstance()

## Always start by initializing Qt (only once per application)
app = QApplication([])


In [1]:
# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file 'ExpertUserUI.ui'
#
# Created by: PyQt5 UI code generator 5.9.2
#
# WARNING! All changes made in this file will be lost!

from PyQt5 import QtCore, QtGui, QtWidgets
exposure_time = 100       #micro seconds min=10us (FIXED)
FPS = 25                    #Frame rate to capture 
gain = 0                   #Sensor gain (FIXED)
image_format = 'Mono8'      #image pixel format. 'Mono8' or 'Bayer' for BGR
width = 1440               #image width in pixels (FIXED)
height = 600                #image height in pixels (FIXED)
Show = 1                    #Show = show the video while loop
Skip_frames = 25            #number of frames to skip before imshow
num_frames = 1000            #number of frames to view
video_record = 0            #1: to record the video, 0: No Record
video_mode = 'MJPG'          #AVI or MJPG 
video_name = 'zdddd.avi'    #Filename to save as

#angle variables
angle = 0                   #rotational angle
rot_x = 0                   #x pos value of final rotated and cropped image
rot_y = 0                   #y pos value of final rotated and cropped image
rot_width = 0               #width of final rotated and cropped image
rot_height = 0              #height of final rotated and cropped image

H = 110             #No. of pixels to select for height of Region Of Interest
blur_value = 3      #value = 3,5 or 7 (ODD).median Blur value determines the accuracy of detection
Delay = 1           #time value in miliseconds. (Fastest) Minimum = 1ms
Show = 1            #To display the image. 1 = On, 0 = Off
Skip_frames = 10     #number of frames to skip before Im showing
Channels = 35       #number of Channels
Export_Results = 1  #export data to excel
camera_height = 0

target_pos_pressure = 200
current_pos_pressure = 0
target_neg_pressure = -300
current_neg_pressure = 0

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(694, 429)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.tabWidget = QtWidgets.QTabWidget(self.centralwidget)
        self.tabWidget.setGeometry(QtCore.QRect(10, 50, 631, 361))
        self.tabWidget.setObjectName("tabWidget")
        
        #tab_2 system check 
        self.tab_2 = QtWidgets.QWidget()
        self.tab_2.setObjectName("tab_2")
        self.tableView = QtWidgets.QTableView(self.tab_2)
        self.tableView.setGeometry(QtCore.QRect(300, 20, 311, 281))
        self.tableView.setObjectName("tableView")
        self.label_17 = QtWidgets.QLabel(self.tab_2)
        self.label_17.setGeometry(QtCore.QRect(420, 40, 71, 21))
        self.label_17.setText("")
        self.label_17.setObjectName("label_17")
        self.pushButton_4 = QtWidgets.QPushButton("SYSTEM SCAN", self.tab_2)
        self.pushButton_4.setGeometry(QtCore.QRect(470, 240, 121, 31))
        self.pushButton_4.setObjectName("pushButton_4")
        self.groupBox = QtWidgets.QGroupBox(self.tab_2)
        self.groupBox.setGeometry(QtCore.QRect(320, 50, 120, 101))
        self.groupBox.setObjectName("groupBox")
        self.label_20 = QtWidgets.QLabel("Stepper Motor",self.groupBox)
        self.label_20.setGeometry(QtCore.QRect(10, 10, 51, 16))
        self.label_20.setObjectName("label_20")
        self.label_21 = QtWidgets.QLabel("Valves",self.groupBox)
        self.label_21.setGeometry(QtCore.QRect(10, 30, 35, 16))
        self.label_21.setObjectName("label_21")
        self.label_22 = QtWidgets.QLabel("Chip Interface",self.groupBox)
        self.label_22.setGeometry(QtCore.QRect(10, 50, 61, 16))
        self.label_22.setObjectName("label_22")
        self.label_23 = QtWidgets.QLabel("Pressure Tight",self.groupBox)
        self.label_23.setGeometry(QtCore.QRect(10, 70, 61, 16))
        self.label_23.setObjectName("label_23")
        self.groupBox_2 = QtWidgets.QGroupBox(self.tab_2)
        self.groupBox_2.setGeometry(QtCore.QRect(470, 50, 120, 101))
        self.groupBox_2.setObjectName("groupBox_2")
        self.label_24 = QtWidgets.QLabel("Stage Stepper",self.groupBox_2)
        self.label_24.setGeometry(QtCore.QRect(10, 20, 51, 16))
        self.label_24.setObjectName("label_24")
        self.label_26 = QtWidgets.QLabel("Camera Stepper", self.groupBox_2)
        self.label_26.setGeometry(QtCore.QRect(10, 40, 61, 16))
        self.label_26.setObjectName("label_26")
        self.label_25 = QtWidgets.QLabel("Camera Output", self.groupBox_2)
        self.label_25.setGeometry(QtCore.QRect(10, 60, 61, 16))
        self.label_25.setObjectName("label_25")
        self.groupBox_4 = QtWidgets.QGroupBox(self.tab_2)
        self.groupBox_4.setGeometry(QtCore.QRect(320, 170, 120, 101))
        self.groupBox_4.setObjectName("groupBox_4")
        self.label_27 = QtWidgets.QLabel("Electronic Feedback",self.groupBox_4)
        self.label_27.setGeometry(QtCore.QRect(10, 20, 51, 31))
        self.label_27.setObjectName("label_27")
        self.label_28 = QtWidgets.QLabel("Barcode Scanner", self.groupBox_4)
        self.label_28.setGeometry(QtCore.QRect(10, 60, 61, 16))
        self.label_28.setObjectName("label_28")
        self.tabWidget.addTab(self.tab_2, "")
        
        #tab_21 camera 
        self.tab_21 = QtWidgets.QWidget()
        self.tab_21.setObjectName("tab_21")
        self.widget = QtWidgets.QWidget(self.tab_21)
        self.widget.setGeometry(QtCore.QRect(10, 40, 331, 211))
        self.widget.setObjectName("widget")
        self.groupBox_3 = QtWidgets.QGroupBox(self.tab_21)
        self.groupBox_3.setGeometry(QtCore.QRect(360, 30, 251, 301))
        self.groupBox_3.setObjectName("groupBox_3")
        self.fps_label = QtWidgets.QLabel('FPS',self.groupBox_3)
        self.fps_label.setGeometry(QtCore.QRect(10, 20, 35, 10))
        self.fps_label.setObjectName("fps_label")
        self.exposure_label = QtWidgets.QLabel('exposure time',self.groupBox_3)
        self.exposure_label.setGeometry(QtCore.QRect(10, 60, 41, 20))
        self.exposure_label.setObjectName("exposure_label")
        
        self.frame_number_label = QtWidgets.QLabel('frame_number',self.groupBox_3)
        self.frame_number_label.setGeometry(QtCore.QRect(130, 20, 41, 16))
        self.frame_number_label.setObjectName("label_5")
        self.gain_label = QtWidgets.QLabel('gain',self.groupBox_3)
        self.gain_label.setGeometry(QtCore.QRect(10, 100, 35, 10))
        self.gain_label.setObjectName("gain_label")
        self.blur_value_label = QtWidgets.QLabel('blur_value',self.groupBox_3)
        self.blur_value_label.setGeometry(QtCore.QRect(130, 60, 35, 20))
        self.blur_value_label.setObjectName("blur_value_label")
        self.delay_label = QtWidgets.QLabel('delay',self.groupBox_3)
        self.delay_label.setGeometry(QtCore.QRect(130, 100, 41, 16))
        self.delay_label.setObjectName("delay_label")
        self.skip_frame_label = QtWidgets.QLabel('skip_frames',self.groupBox_3)
        self.skip_frame_label.setGeometry(QtCore.QRect(130, 140, 35, 16))
        self.skip_frame_label.setObjectName("skip_frame_label")
        
        self.fps_text = QtWidgets.QLineEdit(str(FPS),self.groupBox_3)
        self.fps_text.setGeometry(QtCore.QRect(60, 20, 51, 16))
        self.fps_text.setObjectName("lineEdit")
        self.exposure_text = QtWidgets.QLineEdit(str(exposure_time),self.groupBox_3)
        self.exposure_text.setGeometry(QtCore.QRect(60, 60, 51, 16))
        self.exposure_text.setObjectName("exposure_text")
        self.num_frames_text = QtWidgets.QLineEdit(str(num_frames),self.groupBox_3)
        self.num_frames_text.setGeometry(QtCore.QRect(180, 20, 51, 16))
        self.num_frames_text.setObjectName("num_frames_text")
        self.gain_text = QtWidgets.QLineEdit(str(gain),self.groupBox_3)
        self.gain_text.setGeometry(QtCore.QRect(60, 100, 51, 16))
        self.gain_text.setObjectName("gain_text")
        self.blur_value_text = QtWidgets.QLineEdit(str(blur_value),self.groupBox_3)
        self.blur_value_text.setGeometry(QtCore.QRect(180, 60, 51, 16))
        self.blur_value_text.setObjectName("blur_value_text")
        self.skip_frame_text = QtWidgets.QLineEdit(str(Skip_frames),self.groupBox_3)
        self.skip_frame_text.setGeometry(QtCore.QRect(180, 140, 51, 16))
        self.skip_frame_text.setObjectName("skip_frame_text")
        self.delay_text = QtWidgets.QLineEdit(str(Delay),self.groupBox_3)
        self.delay_text.setGeometry(QtCore.QRect(180, 100, 51, 16))
        self.delay_text.setObjectName("delay_text")
        self.camera_height_label = QtWidgets.QLabel('camera height',self.groupBox_3)
        self.camera_height_label.setGeometry(QtCore.QRect(10, 140, 41, 20))
        self.camera_height_label.setObjectName("camera_height_label")
        self.camera_height_text = QtWidgets.QLineEdit(str(camera_height),self.groupBox_3)
        self.camera_height_text.setGeometry(QtCore.QRect(60, 140, 51, 16))
        self.camera_height_text.setObjectName("camera_height_text")
        self.pushButton_2 = QtWidgets.QPushButton("AUTO FOCUS", self.groupBox_3)
        self.pushButton_2.setGeometry(QtCore.QRect(60, 180, 56, 21))
        self.pushButton_2.setObjectName("pushButton_2")
        self.pushButton_3 = QtWidgets.QPushButton("SAVE",self.groupBox_3)
        self.pushButton_3.setGeometry(QtCore.QRect(130, 180, 101, 21))
        self.pushButton_3.setObjectName("pushButton_3")
        self.tabWidget.addTab(self.tab_21, "")
        
        #tab3 pressure
        self.tab_3 = QtWidgets.QWidget()
        self.tab_3.setObjectName("tab_3")
        self.label_15 = QtWidgets.QLabel("CURRENT +ve PRESSURE VALUE", self.tab_3)
        self.label_15.setGeometry(QtCore.QRect(40, 70, 81, 41))
        self.label_15.setObjectName("label_15")
        self.pushButton_5 = QtWidgets.QPushButton('SAVE',self.tab_3)
        self.pushButton_5.setGeometry(QtCore.QRect(120, 220, 71, 21))
        self.pushButton_5.setObjectName("pushButton_5")
        self.label_29 = QtWidgets.QLabel("TARGET +ve PRESSURE VALUE", self.tab_3)
        self.label_29.setGeometry(QtCore.QRect(40, 30, 81, 41))
        self.label_29.setObjectName("label_29")
        self.label_30 = QtWidgets.QLabel("TARGET -ve PRESSURE VALUE", self.tab_3)
        self.label_30.setGeometry(QtCore.QRect(40, 120, 81, 41))
        self.label_30.setObjectName("label_30")
        self.label_31 = QtWidgets.QLabel("CURRENT -ve PRESSURE VALUE", self.tab_3)
        self.label_31.setGeometry(QtCore.QRect(40, 160, 81, 41))
        self.label_31.setObjectName("label_31")
        self.lineEdit_11 = QtWidgets.QLineEdit(str(target_pos_pressure),self.tab_3)
        self.lineEdit_11.setGeometry(QtCore.QRect(120, 40, 51, 16))
        self.lineEdit_11.setObjectName("lineEdit_11")
        self.lineEdit_12 = QtWidgets.QLineEdit(str(current_pos_pressure),self.tab_3)
        self.lineEdit_12.setGeometry(QtCore.QRect(120, 90, 51, 16))
        self.lineEdit_12.setObjectName("lineEdit_12")
        self.lineEdit_13 = QtWidgets.QLineEdit(str(target_neg_pressure),self.tab_3)
        self.lineEdit_13.setGeometry(QtCore.QRect(120, 130, 51, 16))
        self.lineEdit_13.setObjectName("lineEdit_13")
        self.lineEdit_14 = QtWidgets.QLineEdit(str(current_neg_pressure),self.tab_3)
        self.lineEdit_14.setGeometry(QtCore.QRect(120, 170, 51, 16))
        self.lineEdit_14.setObjectName("lineEdit_14")
        self.pushButton_6 = QtWidgets.QPushButton("PRESSURE CHECK",self.tab_3)
        self.pushButton_6.setGeometry(QtCore.QRect(40, 220, 71, 21))
        self.pushButton_6.setObjectName("pushButton_6")
        self.pushButton_7 = QtWidgets.QPushButton("MANUAL CHECK", self.tab_3)
        self.pushButton_7.setGeometry(QtCore.QRect(40, 260, 71, 21))
        self.pushButton_7.setObjectName("pushButton_7")
        self.tabWidget.addTab(self.tab_3, "")
        
        #tab_4 run and export
        self.tab_4 = QtWidgets.QWidget()
        self.tab_4.setObjectName("tab_4")
        self.label_19 = QtWidgets.QLabel("VIDEO EXPORT SETTINGS",self.tab_4)
        self.label_19.setGeometry(QtCore.QRect(30, 30, 101, 21))
        self.label_19.setObjectName("label_19")
        self.label_57 = QtWidgets.QLabel("DATA EXPORT SETTINGS", self.tab_4)
        self.label_57.setGeometry(QtCore.QRect(150, 30, 101, 21))
        self.label_57.setObjectName("label_57")
        self.label_58 = QtWidgets.QLabel("VIDEO FILE NAME",self.tab_4)
        self.label_58.setGeometry(QtCore.QRect(30, 50, 51, 41))
        self.label_58.setObjectName("label_58")
        self.label_59 = QtWidgets.QLabel("VIDEO TYPE", self.tab_4)
        self.label_59.setGeometry(QtCore.QRect(30, 100, 51, 41))
        self.label_59.setObjectName("label_59")
        self.comboBox_2 = QtWidgets.QComboBox(self.tab_4)
        self.comboBox_2.setGeometry(QtCore.QRect(70, 110, 53, 16))
        self.comboBox_2.setObjectName("comboBox_2")
        self.label_60 = QtWidgets.QLabel("DATA FILE NAME",self.tab_4)
        self.label_60.setGeometry(QtCore.QRect(150, 50, 51, 41))
        self.label_60.setObjectName("label_60")
        self.label_61 = QtWidgets.QLabel("FOLDER",self.tab_4)
        self.label_61.setGeometry(QtCore.QRect(30, 160, 51, 21))
        self.label_61.setObjectName("label_61")
        self.lineEdit_29 = QtWidgets.QLineEdit(self.tab_4)
        self.lineEdit_29.setGeometry(QtCore.QRect(70, 160, 51, 16))
        self.lineEdit_29.setObjectName("lineEdit_29")
        self.lineEdit_30 = QtWidgets.QLineEdit(self.tab_4)
        self.lineEdit_30.setGeometry(QtCore.QRect(180, 60, 51, 16))
        self.lineEdit_30.setObjectName("lineEdit_30")
        self.lineEdit_31 = QtWidgets.QLineEdit(self.tab_4)
        self.lineEdit_31.setGeometry(QtCore.QRect(70, 60, 51, 16))
        self.lineEdit_31.setObjectName("lineEdit_31")
        self.pushButton_14 = QtWidgets.QPushButton("RUN & EXPORT",self.tab_4)
        self.pushButton_14.setGeometry(QtCore.QRect(150, 150, 111, 31))
        self.pushButton_14.setObjectName("pushButton_14")
        self.tabWidget.addTab(self.tab_4, "")
        
        #tab_5 log
        self.tab_5 = QtWidgets.QWidget()
        self.tab_5.setObjectName("tab_5")
        self.LOGTABLE = QtWidgets.QTableView(self.tab_5)
        self.LOGTABLE.setGeometry(QtCore.QRect(60, 60, 431, 231))
        self.LOGTABLE.setObjectName("LOGTABLE")
        self.label_62 = QtWidgets.QLabel("DATE(dd/mm/yy)", self.tab_5)
        self.label_62.setGeometry(QtCore.QRect(100, 40, 71, 16))
        self.label_62.setObjectName("label_62")
        self.label_63 = QtWidgets.QLabel("USER_ID",self.tab_5)
        self.label_63.setGeometry(QtCore.QRect(210, 40, 51, 16))
        self.label_63.setObjectName("label_63")
        self.label_64 = QtWidgets.QLabel("CHANGE_LOG_SUMMARY",self.tab_5)
        self.label_64.setGeometry(QtCore.QRect(280, 40, 161, 16))
        self.label_64.setObjectName("label_64")
        self.tabWidget.addTab(self.tab_5, "")
        self.pushButton = QtWidgets.QPushButton('EXIT', self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(580, 20, 71, 21))
        self.pushButton.setObjectName("pushButton")
        self.groupBox_5 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_5.setGeometry(QtCore.QRect(30, 10, 301, 31))
        self.groupBox_5.setObjectName("groupBox_5")
        self.label = QtWidgets.QLabel("USER_ID", self.groupBox_5)
        self.label.setGeometry(QtCore.QRect(0, 10, 35, 16))
        self.label.setObjectName("label")
        self.textBrowser = QtWidgets.QTextBrowser(self.groupBox_5)
        self.textBrowser.setGeometry(QtCore.QRect(50, 10, 71, 16))
        self.textBrowser.setObjectName("textBrowser")
        self.textBrowser_2 = QtWidgets.QTextBrowser(self.groupBox_5)
        self.textBrowser_2.setGeometry(QtCore.QRect(220, 10, 71, 16))
        self.textBrowser_2.setObjectName("textBrowser_2")
        self.label_2 = QtWidgets.QLabel(self.groupBox_5)
        self.label_2.setGeometry(QtCore.QRect(170, 10, 35, 16))
        self.label_2.setObjectName("label_2")
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        self.tabWidget.setCurrentIndex(1)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.groupBox.setTitle(_translate("MainWindow", "PRESSURE CHECKS"))
        self.groupBox_2.setTitle(_translate("MainWindow", "OPTICS CHECKS"))
        self.groupBox_4.setTitle(_translate("MainWindow", "MISC CHECKS"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tab_2), _translate("MainWindow", "STATUS"))
        self.groupBox_3.setTitle(_translate("MainWindow", "PARAMETERS"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tab_21), _translate("MainWindow", "OPTICS"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tab_3), _translate("MainWindow", "PRESSURE"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tab_4), _translate("MainWindow", "UAT"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.tab_5), _translate("MainWindow", "SYSTEM_LOG"))
        self.groupBox_5.setTitle(_translate("MainWindow", "LOG"))
        self.label_2.setText(_translate("MainWindow", "PRESET"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
#     app.exec_()
    sys.exit(app.exec_())

SystemExit: 0

C:\Anaconda3\envs\heyjody\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
